## Analytical solution

We use the convolution form from Cole's transformation like Basdevant1986. We assume initial conditions $u(x,0) = -\sin{(\pi x)}$. The solution is expressed as
$$u(x,t) = \frac{I_1(x,t)}{I_2(x,t)},$$
where
$$I_1(x,t) = -\int_{-\infty}^{+\infty} \sin(\pi(x-\eta))\, f(x-\eta) \exp\left(-\frac{\eta^2}{4 \nu t}\right)\, \mathrm{d}\eta,$$
$$I_2(x,t) = \int_{-\infty}^{+\infty} f(x-\eta) \exp\left(-\frac{\eta^2}{4 \nu t}\right)\, \mathrm{d}\eta,$$
$$f(y) = \exp{\left(-\frac{\cos{(\pi y})}{2\pi\nu}\right)}.$$
A change of variables $\eta = Sz$ with $S = \sqrt{4 \nu t}$ is used so that
$$d\eta = S \,\mathrm{d}z, \quad \exp\left(-\frac{\eta^2}{4\nu t}\right) = \exp(-z^2),$$
yielding
$$I_1(x,t) = -\int_{-\infty}^{+\infty} \sin(\pi(x-Sz)) f(x-Sz) \exp\left(-z^2\right)\, \mathrm{d}\eta,$$
$$I_2(x,t) = \int_{-\infty}^{+\infty} f(x-Sz) \exp\left(-z^2\right)\, \mathrm{d}\eta,$$
Gauss-Hermite quadrature (with $n$ nodes) can now be used to evaluate these integrals:
$$I_1(x,t) \approx \sum_{i=1}^n w_i \sin(\pi(x-Sz_i))\, f(x-Sz_i)$$
$$I_2(x,t) \approx \sum_{i=1}^n w_i f(x-Sz_i)$$
Here, $z_i$ are the roots of the Hermite polynomial are the corresponding weights $w_i$. I use Julia package *FastGaussQuadrature* to provide the $z_i$ and $w_i$.

